In [1]:
# Load required libraries
import numpy as np
import os
import mdtraj as md 
import itertools
import pandas as pd 
from Bio import PDB 
import warnings #Optional
warnings.filterwarnings("ignore") #Optional

In [2]:
def get_coordinates(conf_name, pdb = None, traj = None, res_start = None, res_end = None, which_chain = None):
    
    aa_list = list(["ALA", "ARG", "ASN", "ASP", "CYS", "GLN", "GLU","GLY","HIS", "ILE", "LEU", "LYS", "MET", "PHE","PRO", "SER", "THR", "TRP", "TYR", "VAL"])
    
    parser = PDB.PDBParser()

    def get_structure(conf_name, conf_path, chain_id = None): 
      
        os.chdir(conf_path)
        struct = parser.get_structure('prot',conf_name)
   
        coor_x=list()
        coor_y=list()
        coor_z=list()
        model_list=list()
        chain_list=list()
        residue_list=list()
        atom_list=list()
        position_list=list()
        
        for model in struct:
            for chain in model:                
                for residue in chain:
                    for atom in residue:
                        x,y,z = atom.get_coord()
                        coor_x.append(x)
                        coor_y.append(y)
                        coor_z.append(z)
                        model_list.append(1+model.id)
                        chain_list.append(chain.id)
                        residue_list.append(residue.get_resname())
                        atom_list.append(atom.id)
                        position_list.append(residue.get_full_id()[3][1])
                                       
        data = {'Model': model_list,
                'Chain': chain_list,
                'Residue': residue_list,
                'Atom': atom_list,
                'Position': position_list,
                'coor_x': coor_x,
                'coor_y': coor_y,
                'coor_z': coor_z
                }
                       
        df = pd.DataFrame (data, columns = ['Model','Chain','Residue','Atom','Position','coor_x','coor_y','coor_z'],index=None)
        df = df[df.Model == df.Model[0]] # Keep just one model
        if chain_id is None:
            df = df[df.Chain == df.Chain[0]] # Keep just one chain
        else:
            df = df[df.Chain == chain_id] # Keep just one chain
        
        # Check for HIE
        df = df.replace('HIE', 'HIS')
                       
        return df
            
    if traj is None and pdb is not None:
 
        df = get_structure(conf_name, conf_path = pdb, chain_id = which_chain)
         # Remove residues not in aa_list
        df = df.loc[df["Residue"].isin(aa_list),]
        L = len(np.unique(df.Position))
        
    elif pdb is None and traj is not None:
        
        traj = traj[conf_name]
        top_table = traj.top.to_dataframe()[0]
        df = pd.concat([top_table, pd.DataFrame(traj.xyz[0], columns = np.array(['x','y','z']))], axis = 1)
        df = df[['segmentID','chainID','resName','name','resSeq','x','y','z']]
        df.columns = ['Model','Chain','Residue','Atom','Position','coor_x','coor_y','coor_z']
        # Remove residues not in aa_list
        df = df.loc[df["Residue"].isin(aa_list),]
        L = len(np.unique(df.Position))
        
        # Correct division by 10
        df.loc[:,['coor_x', 'coor_y', 'coor_z']] = df.loc[:,['coor_x', 'coor_y', 'coor_z']]*10
               
    if res_start is not None:
        
        df = df.loc[df.Position >= res_start]
        L = len(np.unique(df.Position))
        
    if res_end is not None:
        
        df = df.loc[df.Position <= res_end]
        L = len(np.unique(df.Position))
        
    # Build reference systems
 
    basis_angles = np.array([1.917213, 1.921843, 2.493444])
    b = np.array([np.cos(basis_angles[0]), np.cos(basis_angles[1]), np.cos(basis_angles[2])]).T
     
    # 1. Definition of the reference frame on every sequence position

    CA_coor = df.loc[ (df.Atom == 'CA') , ['coor_x','coor_y','coor_z']].to_numpy() # CA coordinates 
    N_coor = df.loc[ (df.Atom == 'N')  , ['coor_x','coor_y','coor_z']].to_numpy() # N coordinates 
    C_coor = df.loc[ (df.Atom == 'C')  , ['coor_x','coor_y','coor_z']].to_numpy() # C coordinates 
    
    N_CA_coor = N_coor - CA_coor; N_CA_coor = N_CA_coor / np.linalg.norm(N_CA_coor, axis = 1)[:, None]
    C_CA_coor = C_coor - CA_coor; C_CA_coor = C_CA_coor / np.linalg.norm(C_CA_coor, axis = 1)[:, None]
    CxN_coor = np.cross(C_CA_coor, N_CA_coor); CxN_coor = CxN_coor / np.linalg.norm(CxN_coor, axis = 1)[:, None]
     
    A_list = np.concatenate([N_CA_coor,C_CA_coor,CxN_coor], axis = 1)
    A_list = np.reshape(A_list, [np.shape(A_list)[0]*3, 3])
    A_list = [A_list[i:(i+3),:] for i in 3*np.arange(np.shape(N_CA_coor)[0])]

    CB_coor = np.linalg.solve(A_list, [b for i in np.arange(len(A_list))]) # Virtual CB coordinates
 
    # Reference frames 
    
    b1_coor = CB_coor / np.linalg.norm(CB_coor, axis = 1)[:, None] # b1 = CA-CB
    CN_coor = N_CA_coor - C_CA_coor # CN
    b2_coor = np.cross(CN_coor, b1_coor); b2_coor = b2_coor / np.linalg.norm(b2_coor, axis = 1)[:, None] # b2 = b1 x CN
    b3_coor = np.cross(b1_coor, b2_coor); b3_coor = b3_coor / np.linalg.norm(b3_coor, axis = 1)[:, None] # b3 = b1 x b2 = CN for a perfect tetrahedron
       
    P_list = np.concatenate([b1_coor, b2_coor, b3_coor], axis = 1)
    P_list = np.reshape(P_list, [np.shape(P_list)[0]*3, 3]).T
    P_list = [P_list[:,i:(i+3)] for i in 3*np.arange(np.shape(b1_coor)[0])]
    P_list = np.linalg.inv(P_list) # Change-of-basis matrix for each position
    
    positions = df.loc[ ((df.Atom =='CB') & (df.Residue!='GLY')) | ((df.Atom =='CA') & (df.Residue=='GLY')), ['coor_x','coor_y','coor_z']]
   
    pos_pairs = np.array(list(itertools.combinations(range(L), 2)))
    P_list_pairs = [P_list[i] for i in pos_pairs[:,0]]
    positions_pairs = positions.to_numpy()[pos_pairs[:,1],:] - positions.to_numpy()[pos_pairs[:,0],:]
    or1_pairs = b1_coor[pos_pairs[:,1],:]
    or2_pairs = b3_coor[pos_pairs[:,1],:]
    
    relative_pairwise_positions = np.einsum('ij,ikj->ik',positions_pairs, P_list_pairs)
    relative_pairwise_or1 = np.einsum('ij,ikj->ik', or1_pairs, P_list_pairs)
    relative_pairwise_or2 = np.einsum('ij,ikj->ik', or2_pairs, P_list_pairs)
        
    aa_seq = df.Residue[df.Atom == 'CA'].to_numpy()
    d = {item: idx for idx, item in enumerate(aa_list)}
    aa_index = np.array([d.get(item) for item in aa_seq])
    aa_pairs = np.concatenate([aa_index[pos_pairs[:,0]][:,None],aa_index[pos_pairs[:,1]][:, None]], axis = 1)
    positions_and_frames = np.concatenate([relative_pairwise_positions, relative_pairwise_or1,
                                           relative_pairwise_or2, aa_pairs], axis = 1)        
    
    return positions_and_frames